In [1]:
# Install required libraries
!pip install librosa transformers

In [2]:
# Define paths to data
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

In [3]:
# Import necessary libraries
import os
import librosa
import pandas as pd
from transformers import BertTokenizer, BertModel


In [4]:
# List files in directories
cha_files = [os.path.join(cha_path, file) for file in os.listdir(cha_path) if file.endswith('.cha')]
mp3_files = [os.path.join(mp3_path, file) for file in os.listdir(mp3_path) if file.endswith('.mp3')]

print(f"Found {len(cha_files)} CHA files and {len(mp3_files)} MP3 files.")

Found 1255 CHA files and 1253 MP3 files.


In [5]:
import os
import re
import random
from collections import defaultdict, Counter

def extract_diagnosis(cha_file):
    """Extracts the diagnosis from a CHAT transcript file."""
    with open(cha_file, 'r') as file:
        content = file.read()
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|', content)
    return match.group(1) if match else None

# Paths to your CHA and MP3 files
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

# Load all CHA files and their diagnoses
cha_files = [f for f in os.listdir(cha_path) if f.endswith('.cha')]
diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in cha_files]

# Group files by diagnosis, ensuring corresponding MP3 exists
files_by_diagnosis = defaultdict(list)
for cha_file, diag in zip(cha_files, diagnoses):
    mp3_file = cha_file.replace('.cha', '.mp3')
    if os.path.exists(os.path.join(mp3_path, mp3_file)):
        files_by_diagnosis[diag].append(cha_file)

# Select up to 500 files, trying to balance across diagnoses
selected_files = []
for diag, files in files_by_diagnosis.items():
    select_count = min(len(files), max(500 // len(files_by_diagnosis), 1))
    selected_files.extend(random.sample(files, select_count))

# Ensure the selection does not exceed 10 if categories were unbalanced
selected_files = random.sample(selected_files, min(500, len(selected_files)))

# Count of selected diagnoses
selected_diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in selected_files]
diagnosis_count = Counter(selected_diagnoses)

# Find corresponding MP3 files
selected_mp3_files = [f.replace('.cha', '.mp3') for f in selected_files]

# Collect full paths for the selected files
cha_files = [os.path.join(cha_path, f) for f in selected_files]
mp3_files = [os.path.join(mp3_path, f) for f in selected_mp3_files]

# Output results
print("Selected CHA files:", cha_files)
print("Selected MP3 files:", mp3_files)
print("Diagnosis counts:", dict(diagnosis_count))


Selected CHA files: ['/kaggle/input/pittcombined/PittCombined/cha/511-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/465-0s.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/244-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/073-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/042-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/466-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/054-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/033-2r.cha', '/kaggle/input/pittcombined/PittCombined/cha/121-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/061-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/209-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/035-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/341-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/504-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/010-1f.cha', '

In [6]:
cha_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in cha_files])
mp3_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in mp3_files])

unmatched_cha = cha_base_names - mp3_base_names
unmatched_mp3 = mp3_base_names - cha_base_names

print("Unmatched CHA files:", unmatched_cha)
print("Unmatched MP3 files:", unmatched_mp3)

Unmatched CHA files: set()
Unmatched MP3 files: set()


In [7]:
import librosa
from transformers import BertTokenizer, BertModel
import soundfile as sf

# Initialize Clinical BERT
tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = BertModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
import os
import librosa
import soundfile as sf

def preprocess_audio(mp3_file_path, output_wav_path):
    # Define output path within the writable directory
    output_wav_path = os.path.join('/kaggle/working', os.path.basename(output_wav_path))
    
    try:
        # Check if the WAV file already exists
        if not os.path.exists(output_wav_path):
            # Convert MP3 to WAV
            y, sr = librosa.load(mp3_file_path, sr=None)
            sf.write(output_wav_path, y, sr)
            print(f"Converted {mp3_file_path} to WAV.")
        else:
            print(f"WAV file already exists: {output_wav_path}")

        # Extract MFCC features
        y, sr = librosa.load(output_wav_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return mfccs.mean(axis=1)
    except Exception as e:
        print(f"An error occurred while processing {mp3_file_path}: {str(e)}")
        return None

# Replace 'mp3_files' with the actual list of mp3 file paths
audio_features = [preprocess_audio(f, f.replace('.mp3', '.wav')) for f in mp3_files if f.endswith('.mp3')]


Converted /kaggle/input/pittcombined/PittCombined/mp3/511-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/465-0s.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/120-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/244-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/073-1.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/042-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/120-2f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/466-1r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/054-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/033-2r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/121-1.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/061-0r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/209-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/035

In [9]:
# Define a function to preprocess text data
def preprocess_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # Insert text cleaning code here as necessary
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze().detach().numpy()

In [10]:
print(cha_files)

['/kaggle/input/pittcombined/PittCombined/cha/511-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/465-0s.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/244-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/073-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/042-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/120-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/466-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/054-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/033-2r.cha', '/kaggle/input/pittcombined/PittCombined/cha/121-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/061-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/209-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/035-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/341-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/504-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/010-1f.cha', '/kaggle/input/pittco

In [11]:
# Process each file
text_embeddings = [preprocess_text(f) for f in cha_files]

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming text_embeddings and audio_features are lists of numpy arrays
# Convert lists to numpy arrays
text_features = np.array(text_embeddings)
audio_features = np.stack(audio_features)

In [13]:
print(text_features)

[[[-9.24983844e-02  1.78870618e-01 -6.64935946e-01 ...  2.09613591e-01
   -3.23770717e-02 -3.27973217e-02]
  [-3.32992941e-01 -1.41751662e-01 -4.41456772e-02 ...  5.66166304e-02
    4.95336503e-01 -5.69179714e-01]
  [-6.51566032e-03 -4.39609811e-02  4.32788670e-01 ...  6.60784483e-01
   -6.53572679e-01  5.53691924e-01]
  ...
  [-4.53612804e-01  5.26964307e-01 -2.84811050e-01 ... -8.31825018e-01
   -5.02861917e-01 -1.00274587e+00]
  [-3.63404065e-01  3.61687750e-01 -2.51487374e-01 ... -2.65366316e-01
    6.36302009e-02 -8.96883368e-01]
  [ 2.11556152e-01 -3.17274570e-01 -8.35146606e-01 ...  7.46398985e-01
    5.16866088e-01  4.73135561e-01]]

 [[-1.39259741e-01  1.07773736e-01 -6.72583640e-01 ...  4.08235818e-01
    2.32114270e-01  6.69766143e-02]
  [-3.72078776e-01  9.03760344e-02  7.56088644e-02 ... -4.73089442e-02
    4.98369008e-01 -4.62366015e-01]
  [ 4.14523296e-02 -1.45551730e-02  4.50641781e-01 ...  7.49563396e-01
   -7.39353061e-01  5.68040013e-01]
  ...
  [-2.22655773e-01  2.5

In [14]:
print(audio_features)

[[-4.7908560e+02  1.1628792e+02  1.4972824e+01 ... -4.2321461e-01
  -1.6097915e+00 -1.0862691e+00]
 [-5.7285358e+02  1.0150505e+02 -9.9266853e+00 ... -6.4322705e+00
   1.1502075e+00  9.9651331e-01]
 [-5.1138544e+02  1.7027957e+02  4.9297493e+01 ...  3.6024694e+00
  -2.4294708e+00 -2.1723204e+00]
 ...
 [-4.0136343e+02  1.5089748e+02  2.8757872e+01 ... -9.6517096e+00
   1.3681322e+00  6.7996640e+00]
 [-5.2435327e+02  1.3672928e+02  4.7623394e+01 ... -1.2013736e+00
  -1.3594029e+00 -1.5653889e-01]
 [-4.8349057e+02  9.5017220e+01  5.0012803e+00 ... -5.2072687e+00
   1.9041822e+00 -2.8323510e+00]]


In [15]:
text_features = np.mean(text_features, axis=1)


In [16]:
print(text_features)

[[-0.09432855 -0.1648833  -0.2828965  ...  0.05541188 -0.05421472
  -0.12069591]
 [-0.10486376 -0.34659356 -0.1585499  ...  0.07264012  0.08237104
   0.02964038]
 [-0.12931003 -0.2734839  -0.17333847 ...  0.10920083  0.0566419
  -0.12839903]
 ...
 [-0.10093521 -0.24285746 -0.22136825 ...  0.03179805  0.00996696
  -0.03905068]
 [-0.11538723 -0.3225181  -0.19248693 ...  0.02436351  0.059473
  -0.13853495]
 [ 0.01023701 -0.11048397 -0.13561897 ...  0.04463599  0.03181081
  -0.07859965]]


In [17]:
# Ensure audio_features is 2D (it should already be if you've extracted features correctly)
if audio_features.ndim > 2:
    audio_features = np.mean(audio_features, axis=1)

In [18]:
# Combine text and audio features
combined_features = np.concatenate([text_features, audio_features], axis=1)

In [19]:
print(combined_features)

[[-0.09432855 -0.1648833  -0.2828965  ... -0.4232146  -1.6097915
  -1.0862691 ]
 [-0.10486376 -0.34659356 -0.1585499  ... -6.4322705   1.1502075
   0.9965133 ]
 [-0.12931003 -0.2734839  -0.17333847 ...  3.6024694  -2.4294708
  -2.1723204 ]
 ...
 [-0.10093521 -0.24285746 -0.22136825 ... -9.65171     1.3681322
   6.799664  ]
 [-0.11538723 -0.3225181  -0.19248693 ... -1.2013736  -1.3594029
  -0.15653889]
 [ 0.01023701 -0.11048397 -0.13561897 ... -5.2072687   1.9041822
  -2.832351  ]]


In [20]:
# Normalize the combined features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)

In [21]:
import re

def extract_diagnostic_code(cha_file):
    # Read the content of the .cha file
    with open(cha_file, 'r') as file:
        content = file.read()

    # Regex to find the diagnosis in the file content based on the updated structure
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|\|', content)
    if match:
        return match.group(1).strip()
    return None

# Assuming cha_files is a list of paths to your .cha files
labels = [extract_diagnostic_code(f) for f in cha_files]

# Example of how you might use this
for label in labels[:10]:  # Print first 10 labels to verify
    print(label)


PossibleAD
PossibleAD
MCI
ProbableAD
Control
Control
MCI
ProbableAD
Control
MCI


In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [23]:
print(encoded_labels)

[5 5 2 7 0 0 2 7 0 2 0 2 0 7 7 7 7 5 0 8 2 5 2 2 5 8 7 2 0 5 8 0 0 5 7 5 2
 7 7 3 2 5 5 7 0 8 0 2 7 7 0 5 2 2 5 2 0 3 8 5 5 8 7 0 8 5 7 0 3 2 3 7 7 7
 2 0 5 0 0 0 0 2 3 0 0 5 5 2 7 2 5 0 2 2 5 2 7 5 2 7 2 2 8 5 0 0 7 8 5 7 5
 4 5 2 7 2 5 7 0 0 7 7 0 8 0 2 5 2 2 7 5 0 0 5 5 5 4 2 8 8 5 7 0 7 7 5 7 6
 7 0 8 7 2 2 2 3 5 2 2 0 5 2 0 5 7 2 0 0 5 0 0 3 5 2 3 5 7 0 0 5 0 2 7 0 6
 7 7 5 5 2 5 5 7 3 0 2 7 4 7 5 7 2 5 4 8 6 8 0 5 5 5 7 8 5 0 8 0 7 7 2 3 7
 2 0 1 2 0 2 3 7 0 0 7 8 6 7 0 2 0 0 5 5 5 5 7 2 0 5 7 2 7 2 7 2 2 7 2 8 2
 3]


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Conv1D, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam

2024-08-14 11:27:28.417735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 11:27:28.417874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 11:27:28.583390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [26]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_features, encoded_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Finding the maximum label value across all datasets to ensure consistency
max_label = max(np.max(y_train), np.max(y_val), np.max(y_test)) + 1  # Plus one because classes are zero-indexed

# One-hot encode the labels with a consistent number of classes across datasets
y_train = to_categorical(y_train, num_classes=max_label)
y_val = to_categorical(y_val, num_classes=max_label)
y_test = to_categorical(y_test, num_classes=max_label)




In [27]:
print("y_test",y_test)

y_test [[0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0

In [28]:
from tensorflow.keras.regularizers import l2

# Model architecture
input_text = Input(shape=(768,))  # Text features from Clinical BERT
input_audio = Input(shape=(13,))  # Audio features, assuming MFCCs with 13 coefficients

# Text pathway
text_dense = Dense(128, activation='relu', kernel_regularizer=l2(0.1))(input_text)
text_out = Dropout(0.2)(text_dense)


In [29]:
from tensorflow.keras.layers import Reshape

# Audio pathway
audio_reshape = Reshape((13, 1))(input_audio)
conv1 = Conv1D(64, kernel_size=3, activation='relu')(audio_reshape)
conv1_bn = BatchNormalization()(conv1)
conv1_pool = GlobalAveragePooling1D()(conv1_bn)
audio_out = Dropout(0.2)(conv1_pool)

In [30]:
# Fusion and output
concatenated = Concatenate()([text_out, audio_out])
dense_layer = Dense(64, activation='relu')(concatenated)
# Assuming y_train has already been one-hot encoded correctly
num_classes = y_train.shape[1]

# Adjust your output layer
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

In [31]:
print(concatenated)

<KerasTensor shape=(None, 192), dtype=float32, sparse=False, name=keras_tensor_9>


In [32]:
from tensorflow.keras.metrics import Precision, Recall

# Compile model
model = Model(inputs=[input_text, input_audio], outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

In [33]:
# Fit model on training data
model.fit([X_train[:, :768], X_train[:, 768:]], y_train, validation_data=([X_val[:, :768], X_val[:, 768:]], y_val), epochs=100, batch_size=32)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 132ms/step - accuracy: 0.1800 - loss: 23.9438 - precision: 0.3528 - recall: 0.0639 - val_accuracy: 0.3462 - val_loss: 21.4380 - val_precision: 0.7500 - val_recall: 0.0577
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4189 - loss: 20.5931 - precision: 0.6435 - recall: 0.0833 - val_accuracy: 0.3462 - val_loss: 18.8771 - val_precision: 0.7143 - val_recall: 0.0962
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6323 - loss: 17.9224 - precision: 0.7323 - recall: 0.1645 - val_accuracy: 0.3654 - val_loss: 16.5743 - val_precision: 0.6250 - val_recall: 0.0962
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6400 - loss: 15.5455 - precision: 0.7662 - recall: 0.2825 - val_accuracy: 0.3462 - val_loss: 14.4624 - val_precision: 0.4118 - val_recall: 0.1346
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6589 - loss: 13.4013 - precision: 0.8068 - recall: 0.3747 - val_accuracy: 0.3846 - val_loss: 12

In [34]:
# Evaluate the model on the test set
evaluation = model.evaluate([X_test[:, :768], X_test[:, 768:]], y_test)
print(f'Accuracy: {evaluation[1]*100:.2f}%, Precision: {evaluation[2]*100:.2f}%, Recall: {evaluation[3]*100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4663 - loss: 2.6645 - precision: 0.4565 - recall: 0.4175 
Accuracy: 48.08%, Precision: 46.81%, Recall: 42.31%
